In [1]:
import ipyparallel as ipp
import numpy

In [2]:
rc = ipp.Client()

In [3]:
rc.ids

[0, 1, 2, 3]

In [4]:
rc

In [5]:
dview = rc[:] # use all engines

In [6]:
dview.block = True

In [7]:
dview.execute('import numpy')

<AsyncResult: execute:finished>

In [8]:
x = dview.apply(lambda: numpy.random.random())
x

[0.9126448826311061,
 0.8975524778403267,
 0.4929300965852278,
 0.09475996453375313]

In [34]:
def calcPi_dumb(N):
    x = numpy.random.random(N)
    y = numpy.random.random(N)
    return ((x**2+y**2)<1).sum()*4/N

calcPi_dumb(1000)

3.16

In [33]:
%time calcPi_dumb(1000)

CPU times: user 611 µs, sys: 721 µs, total: 1.33 ms
Wall time: 722 µs


3.204

In [12]:
def calcPi_inMem(N=100,chunksize=10000):

    def parts(N, chunksize):
        begin = [chunksize]*int(N/chunksize)
        end = N % chunksize
        if end:
            begin += [end]
        return  begin

    sum = 0
    count = 0
    sizes = parts(N, chunksize)
    for s in sizes:
        x = numpy.random.random(s)
        y = numpy.random.random(s)
        sum += ((x**2+y**2)<1).sum()
        count += s
    assert(count == N)
    return sum*4/N



In [32]:
%time calcPi_inMem(500000,10000)

CPU times: user 13.9 ms, sys: 2.3 ms, total: 16.2 ms
Wall time: 14.1 ms


3.142784

In [16]:
dview.map(calcPi_inMem,[int(1000000000/4)]*len(rc.ids))

[3.141520416, 3.141607792, 3.141661104, 3.141637056]